In [1]:
import pandas as pd

In [26]:
#read in file for dataframe
#df = pd.read_stata('CapstoneCoding\ER_Visits.dta') 
df = pd.read_stata('CapstoneCoding/ER_Visits.dta')
df.head()

,DUID,PID,DUPERSID,EVNTIDX,EVENTRN,ERHEVIDX,FFEEIDX,PANEL,MPCDATA,ERDATEYR,...,ERDOF23X,ERDSL23X,ERDWC23X,ERDOT23X,ERDXP23X,ERDTC23X,IMPFLAG,PERWT23F,VARSTR,VARPSU
0,2790032,106,2790032106,2790032106001001,3 ROUND 3,-1,-1,27 PANEL 27,1 HAS MPC DATA,2023,...,0.0,0.0,0.0,0.0,192.52,1822.00,2 COMPLETE MPC DATA,12162.846840,2064,3
1,2790037,101,2790037101,2790037101007201,3 ROUND 3,-1,-1,27 PANEL 27,2 NO MPC DATA,2023,...,0.0,0.0,0.0,0.0,386.75,1361.00,3 FULLY IMPUTED,9676.522242,2104,3
2,2790043,101,2790043101,2790043101003001,4 ROUND 4,-1,-1,27 PANEL 27,1 HAS MPC DATA,2023,...,0.0,0.0,0.0,0.0,288.53,1357.00,2 COMPLETE MPC DATA,29407.390290,2045,3
3,2790043,101,2790043101,2790043101003501,5 ROUND 5,-1,-1,27 PANEL 27,2 NO MPC DATA,2023,...,0.0,0.0,0.0,0.0,8.60,35.18,3 FULLY IMPUTED,29407.390290,2045,3
4,2790043,103,2790043103,2790043103002201,3 ROUND 3,-1,-1,27 PANEL 27,1 HAS MPC DATA,2023,...,0.0,0.0,0.0,0.0,0.00,245.00,3 FULLY IMPUTED,39688.137454,2045,3


In [ ]:
#investigate dataset
df.info()

In [27]:
#create df with select columns
ER_df = df[['DUPERSID','ERDATEMM','ERDMD23X','ERDMR23X','ERDPV23X',
            'ERFMD23X','ERFMR23X','ERFPV23X']].copy()
ER_df.head()

,DUPERSID,ERDATEMM,ERDMD23X,ERDMR23X,ERDPV23X,ERFMD23X,ERFMR23X,ERFPV23X
0,2790032106,1,91.87,0.00,0.00,256.59,0.00,0.00
1,2790037101,1,0.00,385.31,1.44,0.00,196.49,50.12
2,2790043101,6,0.00,286.50,2.03,0.00,574.11,146.46
3,2790043101,8,0.00,8.60,0.00,0.00,500.14,127.59
4,2790043103,1,0.00,0.00,0.00,0.00,0.00,0.00


In [28]:
#change column names from codebook variables
ER_df.rename(columns={'DUPERSID': 'person_id','ERDATEMM': 'month','ERDMD23X':'dr_medicaid',
                      'ERDMR23X': 'dr_medicare',
                      'ERDPV23X': 'dr_private','ERFMD23X':'f_medicaid','ERFMR23X': 'f_medicare',
                      'ERFPV23X' : 'f_private'}, inplace=True)
ER_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4241 entries, 0 to 4240
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   person_id    4241 non-null   object  
 1   month        4241 non-null   category
 2   dr_medicaid  4241 non-null   float64 
 3   dr_medicare  4241 non-null   float64 
 4   dr_private   4241 non-null   float64 
 5   f_medicaid   4241 non-null   float64 
 6   f_medicare   4241 non-null   float64 
 7   f_private    4241 non-null   float64 
dtypes: category(1), float64(6), object(1)
memory usage: 236.6+ KB


In [29]:
#aggregate column of payment
#df['sum_cols'] = df['col1'] + df['col2']
ER_df['medicaid'] = ER_df['dr_medicaid'] + ER_df['f_medicaid']
ER_df['medicare'] = ER_df['dr_medicare'] + ER_df['f_medicaid']
ER_df['private'] = ER_df['dr_private'] + ER_df['f_private']

In [30]:
#drop columns from df
ER_df = ER_df.drop(['dr_medicaid', 'dr_medicare', 'dr_private',
                   'f_medicare', 'f_medicaid', 'f_private'], axis=1)
ER_df.head()

,person_id,month,medicaid,medicare,private
0,2790032106,1,348.46,256.59,0.00
1,2790037101,1,0.00,385.31,51.56
2,2790043101,6,0.00,286.50,148.49
3,2790043101,8,0.00,8.60,127.59
4,2790043103,1,0.00,0.00,0.00


In [31]:
#change month dtype
ER_df['month'] = ER_df['month'].astype(int)
ER_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4241 entries, 0 to 4240
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   person_id  4241 non-null   object 
 1   month      4241 non-null   int64  
 2   medicaid   4241 non-null   float64
 3   medicare   4241 non-null   float64
 4   private    4241 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 165.8+ KB


In [21]:
#read in medical condition
df_condition = pd.read_stata('CapstoneCoding/Medical_Conditionsta/MedicalConditons.dta') 
df_condition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63656 entries, 0 to 63655
Data columns (total 29 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   DUID      63656 non-null  object  
 1   PID       63656 non-null  int16   
 2   DUPERSID  63656 non-null  object  
 3   CONDN     63656 non-null  int8    
 4   CONDIDX   63656 non-null  object  
 5   PANEL     63656 non-null  category
 6   CONDRN    63656 non-null  category
 7   AGEDIAG   63656 non-null  category
 8   CRND1     63656 non-null  category
 9   CRND2     63656 non-null  category
 10  CRND3     63656 non-null  category
 11  CRND4     63656 non-null  category
 12  CRND5     63656 non-null  category
 13  INJURY    63656 non-null  category
 14  ACCDNWRK  63656 non-null  category
 15  ICD10CDX  63656 non-null  object  
 16  CCSR1X    63656 non-null  object  
 17  CCSR2X    63656 non-null  object  
 18  CCSR3X    63656 non-null  object  
 19  CCSR4X    63656 non-null  object  
 20  HHCOND

In [22]:
#select columns
df_condition = df_condition[['DUPERSID','ICD10CDX']].copy()
df_condition.head()

,DUPERSID,ICD10CDX
0,2790002101,E11
1,2790002101,R03
2,2790002101,L30
3,2790002101,E11
4,2790002101,E55


In [23]:
#rename columns
df_condition.rename(columns={'DUPERSID': 'person_id', 'ICD10CDX': 'ICD10'}, inplace=True)
df_condition.head()

,person_id,ICD10
0,2790002101,E11
1,2790002101,R03
2,2790002101,L30
3,2790002101,E11
4,2790002101,E55


In [32]:
#merge 2 df on person_id
ER_df = pd.merge(ER_df, df_condition, on='person_id', how='left')
ER_df.head()

,person_id,month,medicaid,medicare,private,ICD10
0,2790032106,1,348.46,256.59,0.0,-15
1,2790032106,1,348.46,256.59,0.0,E86
2,2790032106,1,348.46,256.59,0.0,A08
3,2790032106,1,348.46,256.59,0.0,J30
4,2790032106,1,348.46,256.59,0.0,R11


In [33]:
#the ICD10 with -15 values are unknown conditions and will be deleted
ER_df = ER_df[ER_df['ICD10'] != "-15"]
#drop duplicates
ER_df.drop_duplicates(inplace=True)
ER_df.head(15)

,person_id,month,medicaid,medicare,private,ICD10
1,2790032106,1,348.46,256.59,0.00,E86
2,2790032106,1,348.46,256.59,0.00,A08
3,2790032106,1,348.46,256.59,0.00,J30
4,2790032106,1,348.46,256.59,0.00,R11
5,2790037101,1,0.00,385.31,51.56,I10
6,2790037101,1,0.00,385.31,51.56,G45
7,2790037101,1,0.00,385.31,51.56,M19
8,2790037101,1,0.00,385.31,51.56,F32
9,2790037101,1,0.00,385.31,51.56,F99
10,2790037101,1,0.00,385.31,51.56,R00


In [ ]:
#check if payment is duplicate for same person_id, but different ICD10
ER_df = ER_df.groupby(['person_id', 'ICD10', 'month']).agg({
    'medicaid' : 'sum',
    'medicare': 'sum',
    'private': 'sum'}).reset_index()

ER_df.head()

,person_id,ICD10,month,medicaid,medicare,private
0,2790032106,A08,1,348.46,256.59,0.00
1,2790032106,E86,1,348.46,256.59,0.00
2,2790032106,J30,1,348.46,256.59,0.00
3,2790032106,R11,1,348.46,256.59,0.00
4,2790037101,B00,1,0.00,385.31,51.56


In [36]:

#ICD codes are causing multiple payments, but pt was seen all in Jan for multiple conditions
ER_df = ER_df.groupby(['person_id', 'month']).agg({
    'medicaid': 'first', 'medicare': 'first','private': 'first',
    'ICD10': lambda x: ', '.join(sorted(set(x)))}).reset_index()
ER_df.head(15)


,person_id,month,medicaid,medicare,private,ICD10
0,2790032106,1,348.46,256.59,0.00,"A08, E86, J30, R11"
1,2790037101,1,0.00,385.31,51.56,"B00, E07, E55, F32, F41, F99, G45, I10, K92, M..."
2,2790043101,6,0.00,286.50,148.49,"A08, I10, I25, I74, J39, J42, K63"
3,2790043101,8,0.00,8.60,127.59,"A08, I10, I25, I74, J39, J42, K63"
4,2790043103,1,0.00,0.00,0.00,"E11, I10"
5,2790045101,4,367.77,367.77,9.37,R07
6,2790046103,2,0.00,0.00,1884.27,"H66, J39"
7,2790050101,12,0.00,0.00,1515.09,"C44, I10, R41"
8,2790050102,1,0.00,476.24,0.00,"A08, E78, G20, H91, J18, K92, N39, R33, R41"
9,2790050102,5,0.00,492.10,49.16,"A08, E78, G20, H91, J18, K92, N39, R33, R41"
